# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

R. Burn  ->  R. Burn  |  ['R. Burn']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']


Arxiv has 78 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.00093


extracting tarball to tmp_2410.00093...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:488: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.00326


extracting tarball to tmp_2410.00326...

 done.
  0: tmp_2410.00326/cas-dc-sample.tex, 762 lines
  2: tmp_2410.00326/doc/elsdoc-cas.tex, 638 lines
Retrieving document from  https://arxiv.org/e-print/2410.00401


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2410.00326/cas-dc-sample.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2410.00401... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00102-b31b1b.svg)](https://arxiv.org/abs/2410.00102) | **APOKASC-3: The Third Joint Spectroscopic and Asteroseismic catalog for Evolved Stars in the Kepler Fields**  |
|| M. H. Pinsonneault, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-10-02*|
|*Comments*| *43 pages, 25 figures, submitted ApJSupp. Comments welcome. Data tables available on request from pinsonneault.1@osu.edu*|
|**Abstract**|            In the third APOKASC catalog, we present data for the complete sample of 15,808 evolved stars with APOGEE spectroscopic parameters and Kepler asteroseismology. We used ten independent asteroseismic analysis techniques and anchor our system on fundamental radii derived from Gaia $L$ and spectroscopic $T_{\rm eff}$. We provide evolutionary state, asteroseismic surface gravity, mass, radius, age, and the spectroscopic and asteroseismic measurements used to derive them for 12,418 stars. This includes 10,036 exceptionally precise measurements, with median fractional uncertainties in \nmax, \dnu, mass, radius and age of 0.6\%, 0.6\%, 3.8\%, 1.8\%, and 11.1\% respectively. We provide more limited data for 1,624 additional stars which either have lower quality data or are outside of our primary calibration domain. Using lower red giant branch (RGB) stars, we find a median age for the chemical thick disk of $9.14 \pm 0.05 ({\rm ran}) \pm 0.9 ({\rm sys})$ Gyr with an age dispersion of 1.1 Gyr, consistent with our error model. We calibrate our red clump (RC) mass loss to derive an age consistent with the lower RGB and provide asymptotic GB and RGB ages for luminous stars. We also find a sharp upper age boundary in the chemical thin disk. We find that scaling relations are precise and accurate on the lower RGB and RC, but they become more model dependent for more luminous giants and break down at the tip of the RGB. We recommend the usage of multiple methods, calibration to a fundamental scale, and the usage of stellar models to interpret frequency spacings.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00136-b31b1b.svg)](https://arxiv.org/abs/2410.00136) | **JWST captures a sudden stellar outburst and inner disk wall destruction**  |
|| C. Xie, et al. -- incl., <mark>A. Kospal</mark> |
|*Appeared on*| *2024-10-02*|
|*Comments*| *16 pages, 7 figures, submitted to ApJ*|
|**Abstract**|            We present JWST/MIRI observations of T~Cha, a highly variable ($\Delta V \sim$3-5\,mag) accreting Sun-like star surrounded by a disk with a large ($\sim 15$\,au) dust gap. We find that the JWST mid-infrared spectrum is signiticantly different from the {\it Spitzer} spectrum obtained 17 years before, where the emission at short wavelengths ($5-10 \mu m$) has decreased by $\sim 2/3$ while at longer wavelengths ($15-25 \mu m$) it increased by up to a factor of $\sim 3$. This 'seesaw' behavior is contemporary with a fairly constant higher optical emission captured by the All Sky Automated Survey. By analyzing and modelling both SEDs, we propose that JWST caught the star during an outburst that destructed the asymmetric inner disk wall responsible for the high optical variability and lower $15-25$\,micron\ emission during the {\it Spitzer} time. The dust mass lost during this outburst is estimated to be comparable ($\sim 1/5$) to the upper limit of the total micron-sized dust mass in the inner disk of T~Cha now. Monitoring this system during possible future outbursts and more observations of its quiescent state will reveal if the inner disk can be replenished or will continue to be depleted and vanish.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00237-b31b1b.svg)](https://arxiv.org/abs/2410.00237) | **Discovering Large-Scale Structure at $2<z<5$ in the C3VO Survey**  |
|| D. Hung, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-10-02*|
|*Comments*| *36 pages, 16 figures, 5 tables, submitted to ApJ*|
|**Abstract**|            The Charting Cluster Construction with VUDS and ORELSE (C3VO) survey is an ongoing imaging and spectroscopic campaign aiming to map out the growth of structure up to $z\sim5$ and was born from the combination of the VIMOS Ultra Deep Survey (VUDS) and the Observations of Redshift Evolution in Large-Scale Environments (ORELSE) survey. As we previously accomplished with the ORELSE survey, we apply our technique known as Voronoi tessellation Monte-Carlo (VMC) mapping to search for serendipitous galaxy overdensities at $2<z<5$ in the three C3VO fields. We also apply the same technique to mock observations of simulated galaxies with properties derived from the GAlaxy Evolution and Assembly (GAEA) semi-analytic model (SAM) in order to judge the effectiveness of our as a function of redshift, total mass, and fraction of spectroscopic redshifts. We find typical completeness and purity values of the order 30-50%, with a strong dependence on mass and redshift, with values as high as $\sim$80% and $\sim$70%, respectively, in the best-case scenario for $\log (M_{z=0}/M_{\odot}) > 14$. In the C3VO fields, we were able to recover many of the previously known structures in the literature as well as find hundreds of new overdensity candidates, once again demonstrating the powerful capabilities of VMC mapping when applied to wide-field optical and infrared galaxy evolution surveys at ever higher redshifts.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00326-b31b1b.svg)](https://arxiv.org/abs/2410.00326) | **Resonant amplitude distribution of the Hilda asteroids and the free-floating planet flyby scenario**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2024-10-02*|
|*Comments*| *22 pages, 10 figures, accepted for publication in Icarus*|
|**Abstract**|            In some recent work, we provided a quantitative explanation for the number asymmetry of Jupiter Trojans by hypothesizing a free-floating planet (FFP) flyby into the Solar System. In support of that explanation, this paper examines the influence of the same FFP flyby on the Hilda asteroids, which orbit stably in the 3:2 mean motion resonance with Jupiter. The observed Hilda population exhibits two distinct resonant patterns: (1) a lack of Hildas with resonant amplitudes < 40 deg. at eccentricities < 0.1; (2) a nearly complete absence of Hildas with amplitudes < 20 deg., regardless of eccentricity. Previous models of Jupiter migration and resonance capture could account for the eccentricity distribution of Hildas but have failed to replicate the unusual absence of those with the smallest resonant amplitudes, which theoretically should be the most stable. Here we report that the FFP flyby can trigger an extremely rapid outward migration of Jupiter, causing a sudden shift in the 3:2 Jovian resonance. Consequently, Hildas with varying eccentricities would have their resonant amplitudes changed by different degrees, leading to the observed resonant patterns. We additionally show that, in our FFP flyby scenario, these patterns are consistently present across different resonant amplitude distributions of primordial Hildas arising from various formation models. We also place constraints on the potential parameters of the FFP, suggesting it should have an eccentricity of 1-1.3 or larger, an inclination up to 30 deg. or higher, and a minimum mass of about 50 Earth masses.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00401-b31b1b.svg)](https://arxiv.org/abs/2410.00401) | **Cosmological constraints using Minkowski functionals from the first year data of the Hyper Suprime-Cam**  |
|| J. Armijo, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-10-02*|
|*Comments*| *8 pages, 5 figures. Submitted to MNRAS*|
|**Abstract**|            We use Minkowski functionals to analyse weak lensing convergence maps from the first-year data release of the Subaru Hyper Suprime-Cam (HSC-Y1) survey. Minkowski functionals provide a description of the morphological properties of a field, capturing the non-Gaussian features of the Universe matter-density distribution. Using simulated catalogs that reproduce survey conditions and encode cosmological information, we emulate Minkowski functionals predictions across a range of cosmological parameters to derive the best-fit from the data. By applying multiple scales cuts, we rigorously mitigate systematic effects, including baryonic feedback and intrinsic alignments. From the analysis, combining constraints of the angular power spectrum and Minkowski functionals, we obtain $S_8 \equiv \sigma_8\sqrt{\Omega_{\rm m}/0.3} = {0.808}_{-0.046}^{+0.033}$ and $\Omega_{\rm m} = {0.293}_{-0.043}^{+0.157}$. These results represent a $40\%$ improvement on the $S_8$ constraints compared to using power spectrum only, and are consistent with previous non-Gaussian statistics analyses of the same dataset. Our study demonstrates the power of Minkowski functionals beyond two-point statistics for constraining and breaking the degeneracy between $\Omega_{\rm m}$ and $\sigma_8$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00106-b31b1b.svg)](https://arxiv.org/abs/2410.00106) | **Metallicity calibrations based on auroral lines from PHANGS-MUSE data**  |
|| M. Brazzini, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-10-02*|
|*Comments*| *20 pages, 14 figures, 6 tables; accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            We present a chemical analysis of selected HII regions from the PHANGS-MUSE nebular catalogue. Our intent is to empirically re-calibrate strong-line diagnostics of gas-phase metallicity, applicable across a wide range of metallicities within nearby star-forming galaxies. To ensure reliable measurements of auroral line fluxes, we carried out a new spectral fitting procedure whereby only restricted wavelength regions around the emission lines of interest are taken into account: this assures a better fit for the stellar continuum. No prior cuts to nebulae luminosity were applied to limit biases in auroral line detections. Ionic abundances of O+, O++, N+, S+, and S++ were estimated by applying the direct method. We integrated the selected PHANGS-MUSE sample with other existing auroral line catalogues, appropriately re-analysed to obtain a homogeneous dataset. This was used to derive strong-line diagnostic calibrations that span from 12+log(O/H) = 7.5 to 8.8. We investigate their dependence on the ionisation parameter and conclude that it is likely the primary cause of the significant scatter observed in these diagnostics. We apply our newly calibrated strong-line diagnostics to the total sample of HII regions from the PHANGS-MUSE nebular catalogue, and we exploit these indirect metallicity estimates to study the radial metallicity gradient within each of the 19 galaxies of the sample. We compare our results with the literature and find good agreement, validating our procedure and findings. With this paper, we release the full catalogue of auroral and nebular line fluxes for the selected HII regions from the PHANGS-MUSE nebular catalogue. This is the first catalogue of direct chemical abundance measurements carried out with PHANGS-MUSE data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.00093-b31b1b.svg)](https://arxiv.org/abs/2410.00093) | **Planetary population synthesis**  |
|| <mark>R. Burn</mark>, C. Mordasini |
|*Appeared on*| *2024-10-02*|
|*Comments*| *To be published in: Handbook of Exoplanets, 2nd Edition, Hans Deeg and Juan Antonio Belmonte (Eds. in Chief), Springer International Publishing AG, part of Springer Nature. 59 pages, 8 figures. This is an update of arXiv:1804.01532*|
|**Abstract**|            The planetary population synthesis method aims at comprehensively testing planet formation theories against observational evidence and providing theoretical sets of planets to help interpret observations and inform instrument development. Recent developments on the theoretical and observational sides are reviewed: First, observational constraints are summarized, then, the work flow of population synthesis and its two main components are presented, which are, global end-to-end models of planetary formation and evolution and probability distributions for the disk initial conditions. Next, the output of four recent population synthesis models is compared in detail and differences and similarities are discussed. The goal is to help the reader understand the assumptions that were made and how they impact the results. Furthermore, future directions of research are identified and the impact of current and future observational programs is discussed. With JWST, evidence on disk and planet compositions emerges. Planet formation models need to prepare for these near-future developments by including self-consistent magnetic wind-driven gas and dust disk evolution, planetary migration, as well as employ hybrid pebble and planetesimal accretion, which are identified as dominant modes of accretion in different mass regimes.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.00102/./Figure6.PDF', 'tmp_2410.00102/./Figure7.PDF', 'tmp_2410.00102/./Figure10.PDF']
copying  tmp_2410.00102/./Figure6.PDF to _build/html/
copying  tmp_2410.00102/./Figure7.PDF to _build/html/
copying  tmp_2410.00102/./Figure10.PDF to _build/html/
exported in  _build/html/2410.00102.md
    + _build/html/tmp_2410.00102/./Figure6.PDF
    + _build/html/tmp_2410.00102/./Figure7.PDF
    + _build/html/tmp_2410.00102/./Figure10.PDF
found figures ['tmp_2410.00136/./Photometry_Time_larger.png', 'tmp_2410.00136/./OverviewBoth.png', 'tmp_2410.00136/./SingleParameters_plusdata.png']
copying  tmp_2410.00136/./Photometry_Time_larger.png to _build/html/
copying  tmp_2410.00136/./OverviewBoth.png to _build/html/
copying  tmp_2410.00136/./SingleParameters_plusdata.png to _build/html/
exported in  _build/html/2410.00136.md
    + _build/html/tmp_2410.00136/./Photometry_Time_larger.png
    + _build/html/tmp_2410.00136/./OverviewBoth.png
    + _build/html/tmp_2410.00136/./SingleP

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mh}[0]{[M/H]}$
$\newcommand{\afe}[0]{[\alpha/Fe] }$
$\newcommand{\afens}[0]{[\alpha/Fe]}$
$\newcommand{\am}[0]{[\alpha/M] }$
$\newcommand{\amns}[0]{[\alpha/M]}$
$\newcommand{\feh}[0]{[Fe/H] }$
$\newcommand{\fehns}[0]{[Fe/H]}$
$\newcommand{\teff}[0]{T_{\rm eff} }$
$\newcommand{\teffns}[0]{T_{\rm eff}}$
$\newcommand{\logg}[0]{\log{g} }$
$\newcommand{\loggns}[0]{\log{g}}$
$\newcommand{\dnu}[0]{\ensuremath{\Delta \nu}}$
$\newcommand{\fdnu}[0]{\ensuremath{f_{\Delta \nu}}}$
$\newcommand{\nmax}[0]{\ensuremath{\nu_{\rm \max}}}$
$\newcommand{\fnmax}[0]{\ensuremath{f_{\nu \max}}}$
$\newcommand{\rsun}[0]{\ensuremath{R_{\odot}}}$
$\newcommand{\msun}[0]{\ensuremath{M_{\odot}}}$
$\newcommand{\kep}[0]{\textsl{Kepler}}$
$\newcommand{\gaia}[0]{\textsl{Gaia}}$
$\newcommand{\jt}[1]{\textcolor{blue}{#1}}$
$\newcommand{\kc}[1]{\textcolor{green}{[K. Cao: #1]}}$
$\newcommand{\jcz}[1]{\textcolor{grey}{#1}}$
$\newcommand{\ams}[1]{\textcolor{orange}{#1}}$
$\newcommand{\RAG}[1]{\textcolor{blue}{#1}}$
$\newcommand{\amscomm}[1]{\textcolor{purple}{#1}}$
$\newcommand{\BM}[1]{\textcolor{purple}{[BM: #1]}}$</div>



<div id="title">

# APOKASC-3: The Third Joint Spectroscopic and Asteroseismic Catalog for Evolved Stars in the $\kep$ Fields

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.00102-b31b1b.svg)](https://arxiv.org/abs/2410.00102)<mark>Appeared on: 2024-10-02</mark> -  _43 pages, 25 figures, submitted ApJSupp. Comments welcome. Data tables available on request from pinsonneault.1@osu.edu_

</div>
<div id="authors">

M. H. Pinsonneault, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** In the third APOKASC catalog, we present data for the complete sample of 15,808 evolved stars with APOGEE spectroscopic parameters and $\kep$ asteroseismology. We used ten independent asteroseismic analysis techniques and anchor our system on fundamental radii derived from $\gaia$ $L$ and spectroscopic $T_{\rm eff}$ .  We provide evolutionary state, asteroseismic surface gravity, mass, radius, age, and the spectroscopic and asteroseismic measurements used to derive them for 12,418 stars. This includes 10,036 exceptionally precise measurements, with median fractional uncertainties in $\nmax$ , $\dnu$ , mass, radius and age of 0.6 \% , 0.6 \% , 3.8 \% , 1.8 \% , and 11.1 \% respectively. We provide more limited data  for 1,624 additional stars which either have lower quality data or are outside of our primary calibration domain. Using lower red giant branch (RGB) stars, we find a median age for the chemical thick disk of $9.14 \pm 0.05 ({\rm ran}) \pm 0.9 ({\rm sys})$ Gyr with an age dispersion of 1.1 Gyr, consistent with our error model. We calibrate our red clump (RC) mass loss to derive an age consistent with the lower RGB and provide asymptotic GB and RGB ages for luminous stars. We also find a sharp upper age boundary in the chemical thin disk. We find that scaling relations are precise and accurate on the lower RGB and RC, but they become more model dependent for more luminous giants and break down at the tip of the RGB. We recommend the usage of multiple methods, calibration to a fundamental scale, and the usage of stellar models to interpret frequency spacings.

</div>

<div id="div_fig1">

<img src="tmp_2410.00102/./Figure6.PDF" alt="Fig6" width="100%"/>

**Figure 6. -** Frequency spacing corrections for RC stars using the Garstec+Mosser (top left), Garstec+White (middle left) and Sharma+White (lower left) models and weights. Differences between Garstec+Mosser and Garstec+White (top right) measure the impact of the assigned weighting scheme. Differences between Garstec+White and Sharma+White measure the impact of the models used. For a minority of stars, the true $\logg$ range of the sample did not correspond to the range in the model grid, which is responsible for some of the structures seen in the BeSPP results. (*fig:rccor*)

</div>
<div id="div_fig2">

<img src="tmp_2410.00102/./Figure7.PDF" alt="Fig7" width="100%"/>

**Figure 7. -** Frequency spacing corrections for RGB stars using the Garstec+Mosser (top left), Garstec+White (middle left), and Sharma+White (lower left) models and weights. Differences between Garstec+Mosser and Garstec+White (top right) measure the impact of the assigned weighting scheme. Differences between Garstec+White and Sharma+White measure the impact of the models used. (*fig:rgbcor*)

</div>
<div id="div_fig3">

<img src="tmp_2410.00102/./Figure10.PDF" alt="Fig10" width="100%"/>

**Figure 10. -** Fractional difference in age between AGB and RGB stars (top) and the total mass loss on the RGB (bottom) as a function of the current mass of the star. Low mass stars experience much more mass loss. (*fig:mdot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.00102"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ilaria}[1]$
$\newcommand{\cyxie}[1]$
$\newcommand{\uat}[2]{\href{http://vocabs.ands.org.au/repository/api/lda/aas/the-unified-astronomy-thesaurus/current/resource.html?uri=http://astrothesaurus.org/uat/#1}{#2  (#1)}}$
$\newcommand{\affilLPL}{\affiliation{Lunar and Planetary Laboratory, The University of Arizona, Tucson, AZ 85721, USA; \url{cyxie@arizona.edu}}}$</div>



<div id="title">

# JWST captures a sudden stellar outburst and inner disk wall destruction

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.00136-b31b1b.svg)](https://arxiv.org/abs/2410.00136)<mark>Appeared on: 2024-10-02</mark> -  _16 pages, 7 figures, submitted to ApJ_

</div>
<div id="authors">

C. Xie, et al.

</div>
<div id="abstract">

**Abstract:** We present JWST/MIRI observations of T Cha, a highly variable ( $\Delta V \sim$ 3-5 mag) accreting Sun-like star surrounded by a disk with a large ( $\sim 15$ au) dust gap. We find that the JWST mid-infrared spectrum is signiticantly different from the $_ Spitzer_$ spectrum obtained 17 years before, where the emission at short wavelengths ( $5-10 \mu m$ ) has decreased by $\sim 2/3$ while at longer wavelengths ( $15-25 \mu m$ ) it increased by up to a factor of $\sim 3$ . This ‘seesaw’ behavior is contemporary with a fairly constant higher optical emission captured by the All Sky Automated Survey. By analyzing and modelling both SEDs, we propose that JWST caught the star during an outburst that destructed the asymmetric inner disk wall responsible for the high optical variability and lower $15-25$ micron emission during the $_ Spitzer_$ time. The dust mass lost during this outburst is estimated to be comparable ( $\sim 1/5$ ) to the upper limit of the total micron-sized dust mass in the inner disk of T Cha now. Monitoring this system during possible future outbursts and more observations of its quiescent state will reveal if the inner disk can be replenished or will continue to be depleted and vanish.

</div>

<div id="div_fig1">

<img src="tmp_2410.00136/./Photometry_Time_larger.png" alt="Fig1" width="100%"/>

**Figure 1. -** Optical and mid-IR light curves of T Cha. The lines and shaded area indicates the date when some of the photometric points we analyzed in this paper were taken. Blue shaded areas indicates the period when T Cha has less variable but brighter optical emission. The `LCOGT and Andicam' is from [Walter, Brown and France (2018)]().
     (*fig:Photometry*)

</div>
<div id="div_fig2">

<img src="tmp_2410.00136/./OverviewBoth.png" alt="Fig3" width="100%"/>

**Figure 3. -** Overview of the SED of T Cha. The grey dased line is 5600 K stellar photosphere emission. The red and blue solid lines are _ Spitzer_/IRS SED and JWST MIRI/MRS spectra, respectively. The two strong lines in the JWST spectrum are [ArII] and [NeII], and the broad features are PAH features, which are discussed in [Bajaj, Pascucci and Gorti (2024)](). The photometric points in optical bands are from [Alcala, Covino and Franchini (1993)](), [Walter, Brown and France (2018)]() and AAVSO, and the IR photometric points are from various papers (see Table \ref{app:tab:ref}). We highlight some points in blue to indicate they are taken contemporary with the JWST observation (see the blue shaded region next to the JWST line in Fig \ref{fig:Photometry}). The near and mid-IR points connected with dotted lines indicate one contemporary observation at JHKLM bands  ([Alcala, Covino and Franchini 1993]()) .
     (*fig:Overview*)

</div>
<div id="div_fig3">

<img src="tmp_2410.00136/./SingleParameters_plusdata.png" alt="Fig7" width="100%"/>

**Figure 7. -** SED changes with each single parameter. When changing one parameter, other parameters are kept fixed as shown in Table \ref{app:tab:Start}. In Panel 3 the optically thick part is between the star and the observer to illustrate how the maximum optical extinction vary with $\delta \phi$. Notice that the initial parameters set here are not exactly the same as our fitted model for _ Spitzer_.
     (*app:fig:single*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.00136"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

205  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
